# Camera Motion

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 200

logs = {
    "Without cropping": "/media/martin/Samsung_T5/data/endoscopic_data/cholec80_frames/pre_processed_5th_frame_log.pkl",
    "With cropping": "/media/martin/Samsung_T5/data/endoscopic_data/cholec80_single_video_frames_cropped/pre_processed_5th_frame_log_new.pkl"
}

for key in logs:
    log = logs[key]
    data_df = pd.read_pickle(log)
    # https://stackoverflow.com/questions/25577352/plotting-cdf-of-a-pandas-series-in-python
    # label_df = pd.read_pickle("/media/martin/Samsung_T5/data/endoscopic_data/cholec80_frames/pre_processed_log_test_train_labels.pkl")

    data_df.duv_mpd = data_df.duv_mpd.round(decimals=2)
    # data_df['duv_mpd_ra'] = data_df.duv_mpd.rolling(25).mean()
    # data_df.duv_mpd_ra = data_df.duv_mpd_ra.round(decimals=2)

    # # for phase>
    # phases = label_df.Phase.unique()

    # Frequency
    # stats_data_df = data_df.groupby('duv_mpd_ra')['duv_mpd_ra'].agg('count').pipe(pd.DataFrame).rename(columns = {'duv_mpd_ra': 'frequency'})
    stats_data_df = data_df.groupby('duv_mpd')['duv_mpd'].agg('count').pipe(pd.DataFrame).rename(columns = {'duv_mpd': 'frequency'})
    # Pdata_df
    stats_data_df['pdata_df'] = stats_data_df['frequency'] / sum(stats_data_df['frequency'])

    stats_data_df.pdata_df = stats_data_df.pdata_df.rolling(5).mean()
    stats_data_df = stats_data_df.reset_index()

    # stats_data_df.plot(x='duv_mpd_ra', y='pdata_df', grid=True)~
    # stats_data_df.plot(x='duv_mpd', y='pdata_df', grid=True)
    plt.plot(stats_data_df.duv_mpd, stats_data_df.pdata_df, label=key)

plt.grid()
plt.legend()
plt.title("Camera Motion Distribution Cholec80")
plt.xlabel("Mean Pairwise Distance duv / pixels")
plt.ylabel("Probability Density")
plt.show()


# Camera Motion by Phase

In [ ]:
# data_df.hist("duv_mpd", bins=500)
from tqdm import tqdm
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 200
import numpy as np
import pandas as pd

target_phase_id = None
method = "loftr"

if target_phase_id is not None:
    motion_pkls = [
        "22_11_09_deep_pre_processed_frame_increment_5_frames_between_clips_5_log_test_train_no_nan.pkl",
        "22_11_09_loftr_pre_processed_frame_increment_5_frames_between_clips_5_log_test_train_no_nan.pkl"
    ]
    label_pkls = [
        "22_11_09_deep_pre_processed_frame_increment_5_frames_between_clips_5_log_test_train_labels_no_nan.pkl",
        "22_11_09_loftr_pre_processed_frame_increment_5_frames_between_clips_5_log_test_train_labels_no_nan.pkl"
    ]
    for motion_pkl, label_pkl in zip(motion_pkls, label_pkls):
        data_df = pd.read_pickle(f"/media/martin/Samsung_T5/data/endoscopic_data/cholec80_single_video_frames_cropped/{motion_pkl}")
        label_df = pd.read_pickle(f"/media/martin/Samsung_T5/data/endoscopic_data/cholec80_single_video_frames_cropped/{label_pkl}")

        # for phase>
        duv_max = 100
        phases = label_df.Phase.unique()

        phase = phases[target_phase_id]
        phase_df = data_df[label_df.Phase == phase]
        phase_df.duv_mpd = phase_df.duv_mpd.apply(lambda x: np.nan if np.isnan(x).any() else x)
        phase_df = phase_df.dropna()
        phase_df = phase_df[phase_df.duv_mpd < duv_max]

        hist, edges = np.histogram(phase_df.duv_mpd, bins=500, density=True)
        plt.plot(edges[:-1], hist, label=phase)

else:
    # also deep!
    motion_pkl = f"22_11_09_{method}_pre_processed_frame_increment_5_frames_between_clips_5_log_test_train_no_nan.pkl"
    label_pkl = f"22_11_09_{method}_pre_processed_frame_increment_5_frames_between_clips_5_log_test_train_labels_no_nan.pkl"

    data_df = pd.read_pickle(f"/media/martin/Samsung_T5/data/endoscopic_data/cholec80_single_video_frames_cropped/{motion_pkl}")
    label_df = pd.read_pickle(f"/media/martin/Samsung_T5/data/endoscopic_data/cholec80_single_video_frames_cropped/{label_pkl}")

    # # drop non-existend videos from labels
    # unique_vid = data_df.vid.unique()
    # label_df 

    # data_df = data_df.dropna()

    # for phase>
    duv_max = 100
    phases = label_df.Phase.unique()

    for phase in tqdm(phases):
        phase_df = data_df[label_df.Phase == phase]
        phase_df.duv_mpd = phase_df.duv_mpd.apply(lambda x: np.nan if np.isnan(x).any() else x)
        phase_df = phase_df.dropna()
        phase_df = phase_df[phase_df.duv_mpd < duv_max]

        hist, edges = np.histogram(phase_df.duv_mpd, bins=500, density=True)
        plt.plot(edges[:-1], hist, label=phase)

plt.ylim([0, 0.2])
plt.xlim([0, duv_max])
if method == "deep":
    plt.title(r"Cholec80 - Camera Motion Distribution by Phase - Resnet34")
if method == "loftr":
    plt.title(r"Cholec80 - Camera Motion Distribution by Phase - LoFTR")
plt.xlabel(r"Mean Pairwise Distance $\Delta$uv / pixels")
plt.ylabel(r"Probability Density / a.u.")
plt.legend()
plt.grid()
plt.show()

In [ ]:
# for phase>
phases = label_df.Phase.unique()

data_df['duv_mpd_ra'] = data_df.duv_mpd.rolling(25).mean()
data_df.duv_mpd_ra = data_df.duv_mpd_ra.round(decimals=2)

for phase in phases:
    phase_data_df = data_df[label_df.Phase == phase]

    # Frequency
    stats_data_df = phase_data_df.groupby('duv_mpd_ra')['duv_mpd_ra'].agg('count').pipe(pd.DataFrame).rename(columns = {'duv_mpd_ra': 'frequency'})
    # Pdata_df
    stats_data_df['pdata_df'] = stats_data_df['frequency'] / sum(stats_data_df['frequency'])

    stats_data_df.pdata_df = stats_data_df.pdata_df.rolling(5).mean()
    stats_data_df = stats_data_df.reset_index()

    # stats_data_df.plot(x='duv_mpd_ra', y='pdata_df', grid=True, label=phase)
    plt.plot(stats_data_df.duv_mpd_ra, stats_data_df.pdata_df, label=phase)

plt.legend()
plt.grid()
plt.show()

# Samples where LoFTR / Deep Estimates Diverge

In [2]:
import os
import numpy as np
import pandas as pd


loft_pkl = "22_11_09_loftr_pre_processed_frame_increment_5_frames_between_clips_5_log_test_train_no_nan.pkl"
deep_pkl = "22_11_09_deep_pre_processed_frame_increment_5_frames_between_clips_5_log_test_train_no_nan.pkl"

prefix = "/media/martin/Samsung_T5/data/endoscopic_data/cholec80_single_video_frames_cropped"
loft_df = pd.read_pickle(os.path.join(prefix, loft_pkl))
deep_df = pd.read_pickle(os.path.join(prefix, deep_pkl))

# remove lists
loft_df.duv_mpd = loft_df.duv_mpd.apply(lambda x: np.nan if np.isnan(x).any() else x)
deep_df.duv_mpd = deep_df.duv_mpd.apply(lambda x: np.nan if np.isnan(x).any() else x)

# maintain length
# loft_df = loft_df.dropna()
# deep_df = deep_df.dropna()

# print("nans (due to failed homography estimate): ", len(deep_df) - len(loft_df))

In [3]:
# compute distance measure (mean pairwise distance)
joint_df = pd.concat([loft_df.duv, deep_df.duv], axis=1, keys=['loftr', 'deep'])
joint_df["duv_mpd"] = joint_df.apply(lambda x: np.linalg.norm(np.array(x.loftr) - np.array(x.deep), axis=1).mean(), axis=1)
joint_df = pd.concat([joint_df.deep, joint_df.loftr, joint_df.duv_mpd, deep_df.folder, deep_df.file], keys=['deep', 'loftr', 'duv_mpd', 'folder', 'file'], axis=1)
joint_df

,deep,loftr,duv_mpd,folder,file
0,"[[-10.359400749206543, 15.15318489074707], [-7...","[[-10.591853141784668, 15.455631256103516], [-...",1.631831,vid_0,frame_0.npy
5,"[[-5.480680465698242, 13.672898292541504], [-1...","[[-5.467750549316406, 15.025012969970703], [-1...",1.915658,vid_0,frame_5.npy
10,"[[-15.268403053283691, 10.214632987976074], [-...","[[-16.57868194580078, 10.796517372131348], [-7...",2.733251,vid_0,frame_10.npy
15,"[[-2.513118267059326, 12.912035942077637], [-0...","[[-3.468137502670288, 13.821577072143555], [0....",0.736271,vid_0,frame_15.npy
20,"[[-1.1067936420440674, 23.887174606323242], [8...","[[-2.836521863937378, 24.184356689453125], [6....",2.387059,vid_0,frame_20.npy
...,...,...,...,...,...
4423870,"[[8.217355728149414, 24.821287155151367], [9.4...","[[-10.41789722442627, -65.0893325805664], [-79...",480.599798,vid_78,frame_85270.npy
4423875,"[[-3.9679057598114014, -11.362704277038574], [...","[[-76.96363067626953, 27.171613693237305], [62...",64.568054,vid_78,frame_85275.npy
4423880,"[[11.653615951538086, 6.477234363555908], [-5....","[[13.291754722595215, 7.9881391525268555], [-7...",4.610570,vid_78,frame_85280.npy
4423885,"[[6.177763938903809, 15.878209114074707], [-19...","[[22.102922439575195, -2.9197864532470703], [-...",17.400869,vid_78,frame_85285.npy


In [ ]:
joint_df = joint_df[joint_df.duv_mpd > 20]
joint_df.duv_mpd.hist(bins=100)


In [43]:
import sys
sys.path.append("/home/martin/Dev/homography_imitation_learning")


import cv2
import torch
from kornia import image_to_tensor, tensor_to_image
from utils.viz import yt_alpha_blend, create_blend_from_four_point_homography
from utils.io import generate_path

import matplotlib.pyplot as plt
import subprocess

remote = "mhuber@headnode"
remote_prefix = "/nfs/home/mhuber/data/endoscopic_data/cholec80_single_video_frames_cropped"
output_prefix = "/tmp"

increment = 5
file_cnt = 0

N = 100
# samples = joint_df[joint_df.duv_mpd < 20].sample(N)
g_df = joint_df[joint_df.duv_mpd > 20]
samples = g_df[g_df.duv_mpd < 100].sample(N)
# samples = joint_df[joint_df.duv_mpd > 20 and joint_df.duv_mpd < 100].sample(N)

output_path = f"{output_prefix}/geq_20_leq_100"
generate_path(output_path)


for idx, row in samples.iterrows():
    frame_idx = int(row.file.split("_")[-1].replace(".npy", ""))
    img = f"frame_{str(frame_idx)}.npy"
    wrp = f"frame_{str(frame_idx+increment)}.npy"
    
    subprocess.run(["scp", f"{remote}:{remote_prefix}/{row.folder}/{img}", output_path])
    subprocess.run(["scp", f"{remote}:{remote_prefix}/{row.folder}/{wrp}", output_path])


    img = np.load(os.path.join(output_path, img))
    wrp = np.load(os.path.join(output_path, wrp))


    img = image_to_tensor(img, keepdim=False).float() / 255.
    wrp = image_to_tensor(wrp, keepdim=False).float() / 255.
    deep_duv = torch.tensor(row.deep).unsqueeze(0)
    loft_duv = torch.tensor(row.loftr).unsqueeze(0)


    iden_blend = yt_alpha_blend(img, wrp) 
    deep_blend = create_blend_from_four_point_homography(img, wrp, deep_duv)
    loft_blend = create_blend_from_four_point_homography(img, wrp, loft_duv)

    iden_blend = cv2.resize((tensor_to_image(iden_blend, keepdim=False) * 255.).astype(np.uint8), (640, 480))
    deep_blend = cv2.resize((tensor_to_image(deep_blend, keepdim=False) * 255.).astype(np.uint8), (640, 480))
    loft_blend = cv2.resize((tensor_to_image(loft_blend, keepdim=False) * 255.).astype(np.uint8), (640, 480))

    blends = np.concatenate([iden_blend, deep_blend, loft_blend], axis=1)

    cv2.imwrite(f"{output_path}/blend_{frame_idx}.png", blends)


Generating path at /tmp/geq_20_leq_100
